#  Final Project Submission
Please fill out:

- Student name: Ethan Kunin
- Student pace: Full Time
- Scheduled project review date/time: March
- Instructor name: James Irving
- Blog post URL: https://github.com/kuninethan95/dsc-phase-1-project

In [ ]:
# import necessary libraries and packages

import os,glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# display where the data is contained

folder = "/Users/ethankunin/Documents/Flatiron/Phase_1/Movie_Project1/dsc-phase-1-project/zippedData/"
os.listdir(folder)

In [ ]:
files = glob.glob(f"{folder}*.csv*")

In [ ]:
# Load in files and display preview

tables = {}
dashes='---'*25

for file in files:
    ## Save a variable-friendly version of the file name
    table_name = file.replace('.csv.gz','').split('/')[-1].replace('.','_')
    print(dashes)
    
    ## Load and preview dataframe
    print(f"Preview of {table_name}")
    tables[table_name] = pd.read_csv(file)
    display(tables[table_name].head(5))
    print()

# Description of each table with unique identifiers

- imdb_title_crew: id's to link crew members with titles

- tmdb_movies - titles and stats from IMDB

- imdb_title_akas - link between id and movie title

- imdb_title_ratings - link between title and IMDB ratings

- imdb_name_basics - name of cast member and id

- imdb_title_basics - movie title, id, and runtime

- tn_movie_budgets - movie title, release date, and earnings/costs

- bom_movie_gross - movie title, studio, and earnings

- imdb_title_principals - link between movie title and cast cast id

In [ ]:
# Link each file to a Pandas dataframe

filepath0 = files[0]
imdb_title_crew = pd.read_csv(filepath0)

In [ ]:
filepath1 = files[1]
tmdb_movies = pd.read_csv(filepath1)

In [ ]:
filepath2 = files[2]
imdb_title_akas = pd.read_csv(filepath2)

In [ ]:
filepath3 = files[3]
imdb_title_ratings = pd.read_csv(filepath3)

In [ ]:
filepath4 = files[4]
imdb_name_basics = pd.read_csv(filepath4)

In [ ]:
filepath5 = files[5]
imdb_title_basics = pd.read_csv(filepath5)

In [ ]:
filepath6 = files[6]
tn_movie_budgets = pd.read_csv(filepath6)

In [ ]:
filepath7 = files[7]
bom_movie_gross = pd.read_csv(filepath7)

In [ ]:
filepath8 = files[8]
imdb_title_principals = pd.read_csv(filepath8)

## Analyze how runtime impacts revenue

In [ ]:
# Merge movie budgets/earnings with titles to display runtime

movie_rt = tn_movie_budgets.merge(imdb_title_basics, left_on='movie', right_on='primary_title', how='inner')

In [ ]:
movie_rt.head()

In [ ]:
movie_rt.info()

# Going to have to turn production_budget/domestic_gross/worldwide_gross into integers
# Only column with significant null values is runtime_minutues, may account for this by imputing the median

### Clean Data
- Sort for commercial release by only including films with production_budgets > $20,000000
- Only use movies from 2010 onwards
- Convert production_budget/domestic_gross/worldwide_gross into integers
- Impute runtime_minutes with either mean/median
- Check for outliers
- Feature engineer short/medium/long
- Drop unnecessary columns

In [ ]:
def col_to_int(df, colm):
    df[colm] = df[colm].map(lambda x: x.replace('$', '')).map(lambda x: x.replace(',', '')).astype('int')
    return df

In [ ]:
col_to_int(movie_rt, 'production_budget')

In [ ]:
col_to_int(movie_rt, 'domestic_gross')

In [ ]:
col_to_int(movie_rt, 'worldwide_gross')

In [ ]:
# Filter out movies with production budgets under $20,000,000
movie_rt = movie_rt.loc[movie_rt['production_budget'] > 20000000]

In [ ]:
# Feature Engineer year column. Not going to use DateTime yet because will want to engineer a seasonal column
# Convert year into an int
movie_rt['year'] = movie_rt['release_date'].map(lambda x: x[-4:])
movie_rt['year'] = movie_rt['year'].astype('int')

In [ ]:
# Slice out movies from before 2010
movie_rt = movie_rt[movie_rt['year'] > 2010]

In [ ]:
# We have 8.2% of movies with null runtime_minutes
(movie_rt['runtime_minutes'].isna().sum())/len(movie_rt)* 100

In [ ]:
# Check for duplicates
# 298 duplicates, upon inspection doesn't look like there's a reason other than inner join
#Drop if they contain the same movie, release date
movie_rt[movie_rt.duplicated(subset=['movie', 'release_date'])]
movie_rt.duplicated(subset=['movie', 'release_date']).sum()
movie_rt.drop

In [36]:
# Clear out movies with runtimes under 80 minutes
#https://screenwriting.io/what-is-a-feature-film/#:~:text=A%20modern%20feature%20is%20typically,than%2040%20minutes%20a%20feature.
movie_rt = movie_rt[movie_rt['runtime_minutes'] > 80]

In [37]:
movie_rt

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,tconst,primary_title,original_title,start_year,runtime_minutes,genres,year
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,tt1298650,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011,136.0,"Action,Adventure,Fantasy",2011
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,tt6565702,Dark Phoenix,Dark Phoenix,2019,113.0,"Action,Adventure,Sci-Fi",2019
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,tt2395427,Avengers: Age of Ultron,Avengers: Age of Ultron,2015,141.0,"Action,Adventure,Sci-Fi",2015
4,7,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,2048134200,tt4154756,Avengers: Infinity War,Avengers: Infinity War,2018,149.0,"Action,Adventure,Sci-Fi",2018
5,9,"Nov 17, 2017",Justice League,300000000,229024295,655945209,tt0974015,Justice League,Justice League,2017,120.0,"Action,Adventure,Fantasy",2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1708,69,"Aug 4, 2017",Kidnap,21000000,30718107,34836080,tt1458169,Kidnap,Kidnap,2017,95.0,"Crime,Drama,Thriller",2017
1711,72,"Dec 9, 2011",Tinker Tailor Soldier Spy,21000000,24149393,81452811,tt1340800,Tinker Tailor Soldier Spy,Tinker Tailor Soldier Spy,2011,122.0,"Drama,Mystery,Thriller",2011
1713,79,"May 6, 2011",The Beaver,21000000,970816,5046038,tt1321860,The Beaver,The Beaver,2011,91.0,Drama,2011
1714,80,"Feb 24, 2017",Bitter Harvest,21000000,557241,606162,tt3182620,Bitter Harvest,Bitter Harvest,2017,103.0,"Drama,Romance,War",2017
